In [1]:
import torch, torch.nn as nn, torch.nn.functional as F
import glob
# from midi_utils import MidiFile, load_midi_files
# from importlib import reload
# reload(midi_utils) # To reflect the changes.

In [2]:
import midi

from importlib import reload
reload(midi) # To reflect the changes.

<module 'midi' from '/home/volod/work/personal/music-gen/midi.py'>

In [3]:
midi_file_paths = glob.glob("./datasets/jazzomat/*.mid")
print("Num of tacks: ", len(midi_file_paths))

midi_files  = []
for f_name in midi_file_paths:
    if (midi_file := midi.MidiFile.from_filepath(f_name, sequence_len=4)) != None and midi_file.has_instrument("piano"):
        midi_files.append(midi_file)

Num of tacks:  456


In [4]:
midi_files_train = midi_files[:int(0.8 * len(midi_files))]
midi_files_test = midi_files[len(midi_files_train):]

In [5]:
midi_files_sample = midi_files_train[:10]

# midi_files_sample[0].has_instrument("piano")

notes = [m_file.notes() for m_file in midi_files_sample]
notes

  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_Anthropology_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_BluesForBlanche_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_Desafinado_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_InAMellowTone_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_Stardust-1_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/ArtPepper_Stardust-2_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/BennyCarter_IGotItBad_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/BennyCarter_It'sAWonderfulWorld-1_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/BennyCarter_It'sAWonderfulWorld-2_FINAL.mid
  # of instruments: 1
No piano notes found for ./datasets/jazzomat/BennyCarter_Ju

[[], [], [], [], [], [], [], [], [], []]

In [6]:
midi_files = [midi_file]


FEATURE_SIZE = 21
HIDDEN_SIZE = 16

# x.shape => (1, FEATURE_SIZE)
eps = 0.01
Wxh = torch.randn((FEATURE_SIZE, HIDDEN_SIZE)) * eps # x @ Wx => (1, HIDDEN_SIZE)
Whh = torch.randn((HIDDEN_SIZE, HIDDEN_SIZE)) * eps
bh = torch.randn((1, HIDDEN_SIZE)) * eps

Why = torch.randn(HIDDEN_SIZE, FEATURE_SIZE) * eps
by = torch.randn(1, FEATURE_SIZE) * eps

hs_init = torch.rand((1, HIDDEN_SIZE)) * eps

def loss_fn(input, target, hs_prev):
    hs, ys = {}, {}
    hs[-1] = torch.clone(hs_prev)

    for i, x in enumerate(input):
        hs[i] =  torch.tanh((x @ Wxh) + (hs[i-1] @ Whh + bh))
        ys[i] = hs[i] @ Why + by

        loss_note = ""

        # loss = ???

    return hs[len(input)-1], ys

for midi_file in midi_files:
    for input, target in midi_file:
        hs, _ = loss_fn(input, target, hs_init)
        print(hs)

TypeError: iter() returned non-iterator of type 'MidiFile'